In [1]:
import numpy as np
import argparse
import os
import imp
import re
import pickle5 as pickle
import datetime
import random
import math
import logging
import copy
import matplotlib.pyplot as plt
import sklearn
import logging
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.neighbors import kneighbors_graph
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn import Parameter

from utils import utils
from utils.readers import InHospitalMortalityReader
from utils.preprocessing import Discretizer, Normalizer
from utils import metrics
from utils import common_utils
from torch.autograd import Function
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

/home/zzj/anaconda3/envs/distcare/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  after removing the cwd from sys.path.


In [2]:
target_dataset = 'PD' 
RANDOM_SEED = 43
np.random.seed(RANDOM_SEED) #numpy
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED) # cpu
torch.cuda.manual_seed(RANDOM_SEED) #gpu
torch.backends.cudnn.deterministic=True # cudnn

# Use CUDA if available
device = torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu')
# print("available device: {}".format(device))
reverse = False
model_name = 'distcare_adversal'

In [3]:
# if reverse:
#     file_name = 'log_file' + '_' + model_name + '_' + target_dataset +  + '_' + 'reverse' + '.log'
# else:
#     file_name = 'log_file' + '_' + model_name + '_' + target_dataset + '.log'

if reverse:
    file_name = 'log_file' + '_' + model_name + '_' + target_dataset + "_only_shared" + '_' + 'reverse' + '.log'
else:
    file_name = 'log_file' + '_' + model_name + '_' + target_dataset + "_only_shared" + '.log'
def get_logger(name, file_name):
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)
    
    # 以下两行是为了在jupyter notebook 中不重复输出日志
    if logger.root.handlers:
        logger.root.handlers[0].setLevel(logging.WARNING)
 
    handler_stdout = logging.StreamHandler()
    handler_stdout.setLevel(logging.INFO)
    handler_stdout.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
    # logger.addHandler(handler_stdout)
 
    handler_file = logging.FileHandler(filename=file_name, mode='w', encoding='utf-8')
    handler_file.setLevel(logging.DEBUG)
    handler_file.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
    logger.addHandler(handler_file)
 
    return logger

logger = get_logger(__name__,file_name)

logger.debug('这是希望输出的debug内容')
logger.info('这是希望输出的info内容')
logger.warning('这是希望输出的warning内容')

## Get source data

In [4]:
def get_n2n_data(x, y, x_len):
    length = len(x)
    assert length == len(y)
    assert length == len(x_len)
    new_x = []
    new_y = []
    new_x_len = []
    for i in range(length):
        for j in range(len(x[i])):
            new_x.append(x[i][:j+1])
            new_y.append(y[i][j])
            new_x_len.append(j+1)
    return new_x, new_y, new_x_len

In [5]:
source_data_path = './data/Challenge/'
small_part = False
arg_timestep = 1.0
batch_size = 256
epochs = 100
all_x_source = pickle.load(open(source_data_path + 'new_x_front_fill.dat', 'rb'))
all_y_source = pickle.load(open(source_data_path + 'new_y_front_fill.dat', 'rb'))
all_names_source = pickle.load(open(source_data_path + 'new_name.dat', 'rb'))
static_source = pickle.load(open(source_data_path + 'new_demo_front_fill.dat', 'rb'))
mask_x_source = pickle.load(open(source_data_path + 'new_mask_x.dat', 'rb'))
mask_demo_source = pickle.load(open(source_data_path + 'new_mask_demo.dat', 'rb'))
all_x_len_source = [len(i) for i in all_x_source]

if target_dataset == 'PD':
    subset_idx_source = [31, 29, 28, 33, 25, 18, 7, 21, 16, 15, 19, 17, 24, 3, 5, 0]
elif target_dataset == 'TJ':
    subset_idx_source = [27, 29, 18, 16, 26, 33, 28, 31, 32, 15, 11, 25, 21, 20, 9, 17, 30, 19]
elif target_dataset == 'HM':
    subset_idx_source = [0, 1, 2, 3, 5, 9, 11, 12, 13, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]

subset_cnt = len(subset_idx_source)
other_idx = []
for i in range(len(all_x_source[0][0])):
    if i not in subset_idx_source:
        other_idx.append(i)

for i in range(len(all_x_source)): #将共同特征移动到最开始，非共同特征移动到末尾
    cur = np.array(all_x_source[i], dtype=float)
    cur_mask = np.array(mask_x_source[i])
    cur_subset = cur[:, subset_idx_source]
    cur_other = cur[:, other_idx]
    cur_mask_subset = cur_mask[:, subset_idx_source]
    cur_mask_other = cur_mask[:, other_idx]
    all_x_source[i] = np.concatenate((cur_subset, cur_other), axis=1).tolist()
    mask_x_source[i] = np.concatenate((cur_mask_subset, cur_mask_other), axis=1).tolist()


train_num_source =int( len(all_x_source) * 0.8) + 1
logger.info(train_num_source)
dev_num_source =int( len(all_x_source) * 0.1) + 1
logger.info(dev_num_source)
test_num_source =int( len(all_x_source) * 0.1)
logger.info(test_num_source)
assert(train_num_source+dev_num_source+test_num_source == len(all_x_source))

train_x_source = []
train_y_source = []
train_names_source = []
train_static_source = []
train_x_len_source = []
train_mask_x_source = []
for idx in range(train_num_source):
    train_x_source.append(all_x_source[idx])
    train_y_source.append(int(all_y_source[idx][-1]))
    train_names_source.append(all_names_source[idx])
    train_static_source.append(static_source[idx])
    train_x_len_source.append(all_x_len_source[idx])
    train_mask_x_source.append(mask_x_source[idx])

dev_x_source = []
dev_y_source = []
dev_names_source = []
dev_static_source = []
dev_x_len_source = []
dev_mask_x_source = []
for idx in range(train_num_source, train_num_source + dev_num_source):
    dev_x_source.append(all_x_source[idx])
    dev_y_source.append(int(all_y_source[idx][-1]))
    dev_names_source.append(all_names_source[idx])
    dev_static_source.append(static_source[idx])
    dev_x_len_source.append(all_x_len_source[idx])
    dev_mask_x_source.append(mask_x_source[idx])


test_x = []
test_y = []
test_names = []
test_static = []
test_x_len = []
test_mask_x = []
for idx in range(train_num_source + dev_num_source, train_num_source + dev_num_source + test_num_source):
    test_x.append(all_x_source[idx])
    test_y.append(int(all_y_source[idx][-1]))
    test_names.append(all_names_source[idx])
    test_static.append(static_source[idx])
    test_x_len.append(all_x_len_source[idx])
    test_mask_x.append(mask_x_source[idx])


assert(len(train_x_source) == train_num_source)
assert(len(dev_x_source) == dev_num_source)
assert(len(test_x) == test_num_source)

long_x_source = all_x_source
long_y_source = [y[-1] for y in all_y_source]


In [6]:
def get_loss(y_pred, y_true):
    loss = torch.nn.BCELoss()
    return loss(y_pred, y_true)

In [7]:
def get_re_loss(y_pred, y_true):
    loss = torch.nn.MSELoss()
    return loss(y_pred, y_true)

In [8]:
def get_kl_loss(x_pred, x_target):
    loss = torch.nn.KLDivLoss(reduce=True, size_average=True)
    return loss(x_pred, x_target)

In [9]:
def get_wass_dist(x_pred, x_target):
    m1 = torch.mean(x_pred, dim=0)
    m2 = torch.mean(x_target, dim=0)
    v1 = torch.var(x_pred, dim=0)
    v2 = torch.var(x_target, dim=0)
    p1 = torch.sum(torch.pow((m1 - m2), 2))
    p2 = torch.sum(torch.pow(torch.pow(v1, 1/2) - torch.pow(v2, 1/2), 2))
    return torch.pow(p1+p2, 1/2)

In [10]:
def pad_sents(sents, pad_token):
#     print(f'len(pad_token) is {len(pad_token)}')
#     print(f'sents is {sents}')

    sents_padded = []

    max_length = max([len(_) for _ in sents])
    for i in sents:
        padded = list(i) + [pad_token]*(max_length-len(i))
#         print(f'padded is {padded}')
        sents_padded.append(np.array(padded))


    return np.array(sents_padded)

In [11]:
def batch_iter(x, y, lens, batch_size, shuffle=False):
    """ Yield batches of source and target sentences reverse sorted by length (largest to smallest).
    @param data (list of (src_sent, tgt_sent)): list of tuples containing source and target sentence
    @param batch_size (int): batch size
    @param shuffle (boolean): whether to randomly shuffle the dataset
    """
    # batch_num = math.ceil(len(x) / batch_size) # 向下取整
    batch_num = len(x) // batch_size if len(x) % batch_size == 0 else len(x) // batch_size + 1
    # print(f"len(x) is {len(x)}, len(y) is {len(y)}, len(lens) is {len(lens)}, batch_size is {batch_size}, batch_num is {batch_num}")
    index_array = list(range(len(x)))

    if shuffle:
        np.random.shuffle(index_array)

    for i in range(batch_num):
        if (i + 1) * batch_size  < len(x):
            indices = index_array[i * batch_size: (i + 1) * batch_size] #  fetch out all the induces
        else:
            indices = index_array[i * batch_size: ]
        examples = []
        for idx in indices:
            examples.append((x[idx], y[idx],lens[idx]))
       
        examples = sorted(examples, key=lambda e: len(e[0]), reverse=True)
    
        batch_x = [e[0] for e in examples]
        batch_y = [e[1] for e in examples]
        batch_lens = [e[2] for e in examples]

        yield batch_x, batch_y, batch_lens

In [12]:
def length_to_mask(length, max_len=None, dtype=None):
    """length: B.
    return B x max_len.
    If max_len is None, then max of length will be used.
    """
    assert len(length.shape) == 1, 'Length shape should be 1 dimensional.'
    max_len = max_len or length.max().item()
    mask = torch.arange(max_len, device=length.device,
                        dtype=length.dtype).expand(len(length), max_len) < length.unsqueeze(1)
    if dtype is not None:
        mask = torch.as_tensor(mask, dtype=dtype, device=length.device)
    return mask

In [13]:
class SingleAttention(nn.Module):
    def __init__(self, attention_input_dim, attention_hidden_dim, attention_type='add', demographic_dim=12, time_aware=False, use_demographic=False):
        super(SingleAttention, self).__init__()
        
        self.attention_type = attention_type
        self.attention_hidden_dim = attention_hidden_dim
        self.attention_input_dim = attention_input_dim
        self.use_demographic = use_demographic
        self.demographic_dim = demographic_dim
        self.time_aware = time_aware

        # batch_time = torch.arange(0, batch_mask.size()[1], dtype=torch.float32).reshape(1, batch_mask.size()[1], 1)
        # batch_time = batch_time.repeat(batch_mask.size()[0], 1, 1)
        
        if attention_type == 'add':
            if self.time_aware == True:
                # self.Wx = nn.Parameter(torch.randn(attention_input_dim+1, attention_hidden_dim))
                self.Wx = nn.Parameter(torch.randn(attention_input_dim, attention_hidden_dim))
                self.Wtime_aware = nn.Parameter(torch.randn(1, attention_hidden_dim))
                nn.init.kaiming_uniform_(self.Wtime_aware, a=math.sqrt(5))
            else:
                self.Wx = nn.Parameter(torch.randn(attention_input_dim, attention_hidden_dim))
            self.Wt = nn.Parameter(torch.randn(attention_input_dim, attention_hidden_dim))
            self.Wd = nn.Parameter(torch.randn(demographic_dim, attention_hidden_dim))
            self.bh = nn.Parameter(torch.zeros(attention_hidden_dim,))
            self.Wa = nn.Parameter(torch.randn(attention_hidden_dim, 1))
            self.ba = nn.Parameter(torch.zeros(1,))
            
            nn.init.kaiming_uniform_(self.Wd, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wx, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wt, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        elif attention_type == 'mul':
            self.Wa = nn.Parameter(torch.randn(attention_input_dim, attention_input_dim))
            self.ba = nn.Parameter(torch.zeros(1,))
            
            nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        elif attention_type == 'concat':
            if self.time_aware == True:
                self.Wh = nn.Parameter(torch.randn(2*attention_input_dim+1, attention_hidden_dim))
            else:
                self.Wh = nn.Parameter(torch.randn(2*attention_input_dim, attention_hidden_dim))

            self.Wa = nn.Parameter(torch.randn(attention_hidden_dim, 1))
            self.ba = nn.Parameter(torch.zeros(1,))
            
            nn.init.kaiming_uniform_(self.Wh, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        else:
            raise RuntimeError('Wrong attention type.')
        
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax()
    
    def forward(self, input, demo=None):
 
        batch_size, time_step, input_dim = input.size() # batch_size * time_step * hidden_dim(i)
        time_decays = torch.tensor(range(time_step-1,-1,-1), dtype=torch.float32).unsqueeze(-1).unsqueeze(0).to(device)# 1*t*1
        b_time_decays = time_decays.repeat(batch_size,1,1)# b t 1
        
        if self.attention_type == 'add': #B*T*I  @ H*I
            q = torch.matmul(input[:,-1,:], self.Wt)# b h
            q = torch.reshape(q, (batch_size, 1, self.attention_hidden_dim)) #B*1*H
            if self.time_aware == True:
                # k_input = torch.cat((input, time), dim=-1)
                k = torch.matmul(input, self.Wx)#b t h
                # k = torch.reshape(k, (batch_size, 1, time_step, self.attention_hidden_dim)) #B*1*T*H
                time_hidden = torch.matmul(b_time_decays, self.Wtime_aware)#  b t h
            else:
                k = torch.matmul(input, self.Wx)# b t h
                # k = torch.reshape(k, (batch_size, 1, time_step, self.attention_hidden_dim)) #B*1*T*H
            if self.use_demographic == True:
                d = torch.matmul(demo, self.Wd) #B*H
                d = torch.reshape(d, (batch_size, 1, self.attention_hidden_dim)) # b 1 h
            h = q + k + self.bh # b t h
            if self.time_aware == True:
                h += time_hidden
            h = self.tanh(h) #B*T*H
            e = torch.matmul(h, self.Wa) + self.ba #B*T*1
            e = torch.reshape(e, (batch_size, time_step))# b t
        elif self.attention_type == 'mul':
            e = torch.matmul(input[:,-1,:], self.Wa)#b i
            e = torch.matmul(e.unsqueeze(1), input.permute(0,2,1)).squeeze() + self.ba #b t
        elif self.attention_type == 'concat':
            q = input[:,-1,:].unsqueeze(1).repeat(1,time_step,1)# b t i
            k = input
            c = torch.cat((q, k), dim=-1) #B*T*2I
            if self.time_aware == True:
                c = torch.cat((c, b_time_decays), dim=-1) #B*T*2I+1
            h = torch.matmul(c, self.Wh)
            h = self.tanh(h)
            e = torch.matmul(h, self.Wa) + self.ba #B*T*1
            e = torch.reshape(e, (batch_size, time_step)) # b t 
        
        a = self.softmax(e) #B*T
        v = torch.matmul(a.unsqueeze(1), input).squeeze() #B*I

        return v, a

class FinalAttentionQKV(nn.Module):
    def __init__(self, attention_input_dim, attention_hidden_dim, attention_type='add', dropout=None):
        super(FinalAttentionQKV, self).__init__()
        
        self.attention_type = attention_type
        self.attention_hidden_dim = attention_hidden_dim
        self.attention_input_dim = attention_input_dim


        self.W_q = nn.Linear(attention_input_dim, attention_hidden_dim)
        self.W_k = nn.Linear(attention_input_dim, attention_hidden_dim)
        self.W_v = nn.Linear(attention_input_dim, attention_hidden_dim)

        self.W_out = nn.Linear(attention_hidden_dim, 1)

        self.b_in = nn.Parameter(torch.zeros(1,))
        self.b_out = nn.Parameter(torch.zeros(1,))

        nn.init.kaiming_uniform_(self.W_q.weight, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.W_k.weight, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.W_v.weight, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.W_out.weight, a=math.sqrt(5))

        self.Wh = nn.Parameter(torch.randn(2*attention_input_dim, attention_hidden_dim))
        self.Wa = nn.Parameter(torch.randn(attention_hidden_dim, 1))
        self.ba = nn.Parameter(torch.zeros(1,))
        
        nn.init.kaiming_uniform_(self.Wh, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        
        self.dropout = nn.Dropout(p=dropout)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input):
 
        batch_size, time_step, input_dim = input.size() # batch_size * input_dim + 1 * hidden_dim(i)
        input_q = self.W_q(torch.mean(input, dim=1)) # b h
        input_k = self.W_k(input)# b t h
        input_v = self.W_v(input)# b t h

        if self.attention_type == 'add': #B*T*I  @ H*I

            q = torch.reshape(input_q, (batch_size, 1, self.attention_hidden_dim)) #B*1*H
            h = q + input_k + self.b_in # b t h
            h = self.tanh(h) #B*T*H
            e = self.W_out(h) # b t 1
            e = torch.reshape(e, (batch_size, time_step))# b t

        elif self.attention_type == 'mul':
            q = torch.reshape(input_q, (batch_size, self.attention_hidden_dim, 1)) #B*h 1
            e = torch.matmul(input_k, q).squeeze()#b t
            
        elif self.attention_type == 'concat':
            q = input_q.unsqueeze(1).repeat(1,time_step,1)# b t h
            k = input_k
            c = torch.cat((q, k), dim=-1) #B*T*2I
            h = torch.matmul(c, self.Wh)
            h = self.tanh(h)
            e = torch.matmul(h, self.Wa) + self.ba #B*T*1
            e = torch.reshape(e, (batch_size, time_step)) # b t 
        
        a = self.softmax(e) #B*T
        if self.dropout is not None:
            a = self.dropout(a)
        v = torch.matmul(a.unsqueeze(1), input_v).squeeze() #B*I

        return v, a

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

def tile(a, dim, n_tile):
    init_dim = a.size(dim)
    repeat_idx = [1] * a.dim()
    repeat_idx[dim] = n_tile
    a = a.repeat(*(repeat_idx))
    order_index = torch.LongTensor(np.concatenate([init_dim * np.arange(n_tile) + i for i in range(init_dim)])).to(device)
    return torch.index_select(a, dim, order_index).to(device)

class PositionwiseFeedForward(nn.Module): # new added
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x)))), None

    
class PositionalEncoding(nn.Module): # new added / not use anymore
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=400):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0., max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0., d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0 

def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)# b h t d_k
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k) # b h t t
    if mask is not None:# 1 1 t t
        scores = scores.masked_fill(mask == 0, -1e9)# b h t t 
    p_attn = F.softmax(scores, dim = -1)# b h t t
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn # b h t v (d_k) 
    
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, self.d_k * self.h), 3)
        self.final_linear = nn.Linear(d_model, d_model)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1) # 1 1 t t

        nbatches = query.size(0)# b
        input_dim = query.size(1)# i+1
        feature_dim = query.size(-1)# i+1

        #input size -> # batch_size * d_input * hidden_dim
        
        # d_model => h * d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))] # b num_head d_input d_k
        
       
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)# b num_head d_input d_v (d_k) 
        
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)# batch_size * d_input * hidden_dim

        #DeCov 
        DeCov_contexts = x.transpose(0, 1).transpose(1, 2) # I+1 H B
#         print(DeCov_contexts.shape)
        Covs = cov(DeCov_contexts[0,:,:])
        DeCov_loss = 0.5 * (torch.norm(Covs, p = 'fro')**2 - torch.norm(torch.diag(Covs))**2 ) 
        for i in range(11 -1):
            Covs = cov(DeCov_contexts[i+1,:,:])
            DeCov_loss += 0.5 * (torch.norm(Covs, p = 'fro')**2 - torch.norm(torch.diag(Covs))**2 ) 


        return self.final_linear(x), DeCov_loss

class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-7):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

def cov(m, y=None):
    if y is not None:
        m = torch.cat((m, y), dim=0)
    m_exp = torch.mean(m, dim=1)
    x = m - m_exp[:, None]
    cov = 1 / (x.size(1) - 1) * x.mm(x.t())
    return cov

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        returned_value = sublayer(self.norm(x))
        return x + self.dropout(returned_value[0]) , returned_value[1]

class distcare_teacher(nn.Module):
    def __init__(self, input_dim, hidden_dim, d_model,  MHD_num_head, d_ff, output_dim, keep_prob=0.6):
        super(distcare_teacher, self).__init__()

        # hyperparameters
        self.input_dim = input_dim  
        self.hidden_dim = hidden_dim  # d_model
        self.d_model = d_model
        self.MHD_num_head = MHD_num_head
        self.d_ff = d_ff
        self.output_dim = output_dim
        self.keep_prob = keep_prob

        # layers
        self.PositionalEncoding = PositionalEncoding(self.d_model, dropout = 0, max_len = 400)

        self.GRUs = clones(nn.GRU(1, self.hidden_dim, batch_first = True), self.input_dim)
        self.LastStepAttentions = clones(SingleAttention(self.hidden_dim, 8, attention_type='concat', demographic_dim=12, time_aware=True, use_demographic=False),self.input_dim)
        
        self.FinalAttentionQKV = FinalAttentionQKV(self.hidden_dim, self.hidden_dim, attention_type='mul',dropout = 1 - self.keep_prob)

        self.MultiHeadedAttention = MultiHeadedAttention(self.MHD_num_head, self.d_model,dropout = 1 - self.keep_prob)
        self.SublayerConnection = SublayerConnection(self.d_model, dropout = 1 - self.keep_prob)

        self.PositionwiseFeedForward = PositionwiseFeedForward(self.d_model, self.d_ff, dropout=0.1)

        self.demo_proj_main = nn.Linear(12, self.hidden_dim)
        self.demo_proj = nn.Linear(12, self.hidden_dim)
        self.Linear = nn.Linear(self.hidden_dim, 1)
        self.output = nn.Linear(self.input_dim, self.output_dim)

        self.dropout = nn.Dropout(p = 1 - self.keep_prob)
        self.tanh=nn.Tanh()
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.relu=nn.ReLU()

    def forward(self, input, lens):
        lens = lens.to('cpu')
        batch_size = input.size(0)
        time_step = input.size(1)
        feature_dim = input.size(2)
        assert(feature_dim == self.input_dim)# input Tensor : 256 * 48 * 76
        assert(self.d_model % self.MHD_num_head == 0)

        GRU_embeded_input = self.GRUs[0](pack_padded_sequence(input[:,:,0].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
        for i in range(feature_dim-1):
            embeded_input = self.GRUs[i+1](pack_padded_sequence(input[:,:,i+1].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
            GRU_embeded_input = torch.cat((GRU_embeded_input, embeded_input), 1)
        posi_input = self.dropout(GRU_embeded_input) # batch_size * d_input * hidden_dim
        contexts = self.Linear(posi_input).squeeze()# b i
        output = self.output(self.dropout(contexts))# b 1
        output = self.sigmoid(output)
          
        return output, None, contexts
    #, self.MultiHeadedAttention.attn




In [14]:
epochs = 150
batch_size = 256
input_dim = 34
hidden_dim = 32
d_model = 32
MHD_num_head = 4
d_ff = 64
output_dim = 1

model = distcare_teacher(input_dim = input_dim, hidden_dim = hidden_dim, d_model=d_model, MHD_num_head=MHD_num_head, d_ff=d_ff, output_dim = output_dim).to(device)
# input_dim, d_model, d_k, d_v, MHD_num_head, d_ff, output_dim
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [15]:
# Training Teacher
# If you don't want to train Teacher Model:
# - The pretrained taecher model is in direcrtory './model/', and can be directly loaded, 
# - Simply skip this cell and load the model to validate on Dev Dataset.
# logger.info('Training Teacher')

# total_train_loss = []
# total_valid_loss = []
# global_best = 0
# auroc = []
# auprc = []
# minpse = []
# history = []

# pad_token = np.zeros(input_dim)
# # begin_time = time.time()
# best_auroc = 0
# best_auprc = 0
# best_minpse = 0
    
# if target_dataset == 'TJ':    
#     file_name = './model/pretrained-challenge-front-fill-teacher-2covid'
# elif target_dataset == 'HM':
#     file_name = './model/pretrained-challenge-front-fill-teacher-2spain'
# elif target_dataset == 'PD':  
#     file_name = './model/pretrained-challenge-front-fill-teacher-2pd'

# for each_epoch in range(epochs):

#     epoch_loss = []
#     counter_batch = 0
#     model.train()  

#     for step, (batch_x, batch_y, batch_mask_x, batch_lens) in enumerate(batch_iter(train_x, train_y, train_mask_x, train_x_len, batch_size, shuffle=True)):  
#         optimizer.zero_grad()
#         batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
#         batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
#         batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
#         batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)

# #        masks = length_to_mask(batch_lens).unsqueeze(-1).float()

#         opt, decov_loss, emb = model(batch_x, batch_lens)

#         BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1)) # b t 1
# #             REC_Loss = F.mse_loss(masks * recon, masks * batch_x, reduction='mean').to(device)

#         loss = BCE_Loss #+ 1000 * decov_loss

#         epoch_loss.append(BCE_Loss.cpu().detach().numpy())
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 20)
#         optimizer.step()

#         if step % 20 == 0:
#             print('Epoch %d Batch %d: Train Loss = %.4f'%(each_epoch, step, loss.cpu().detach().numpy()))
#             logger.info('Epoch %d Batch %d: Train Loss = %.4f'%(each_epoch, step, loss.cpu().detach().numpy()))

#     epoch_loss = np.mean(epoch_loss)
#     total_train_loss.append(epoch_loss)

#     #Validation
#     y_true = []
#     y_pred = []
#     with torch.no_grad():
#         model.eval()
#         valid_loss = []
#         valid_true = []
#         valid_pred = []
#         for batch_x, batch_y, batch_mask_x, batch_lens in batch_iter(dev_x, dev_y, dev_mask_x, dev_x_len, batch_size):
#             batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
#             batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
#             batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
#             batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)
# #            masks = length_to_mask(batch_lens).unsqueeze(-1).float()


#             opt, decov_loss, emb = model(batch_x, batch_lens)

#             BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))
# #                 REC_Loss = F.mse_loss(recon, batch_x, reduction='mean').to(device)

#             valid_loss.append(BCE_Loss.cpu().detach().numpy())

#             y_pred += list(opt.cpu().detach().numpy().flatten())
#             y_true += list(batch_y.cpu().numpy().flatten())

#         valid_loss = np.mean(valid_loss)
#         total_valid_loss.append(valid_loss)
#         ret = metrics.print_metrics_binary(y_true, y_pred,verbose = 0)
#         history.append(ret)
#         #print()

#         print('Epoch %d: Loss = %.4f Valid loss = %.4f roc = %.4f'%(each_epoch, total_train_loss[-1], total_valid_loss[-1], ret['auroc']))
#         logger.info('Epoch %d: Loss = %.4f Valid loss = %.4f roc = %.4f'%(each_epoch, total_train_loss[-1], total_valid_loss[-1], ret['auroc']))
#         metrics.print_metrics_binary(y_true, y_pred)

#         cur_auroc = ret['auroc']
#         if cur_auroc > best_auroc:
#             best_auroc = cur_auroc
#             best_auprc = ret['auprc']
#             best_minpse = ret['minpse']
#             state = {
#                 'net': model.state_dict(),
#                 'optimizer': optimizer.state_dict(),
#                 'epoch': each_epoch
#             }
#             torch.save(state, file_name)
#             print('------------ Save best model - AUROC: %.4f ------------'%cur_auroc)       

# print('auroc %.4f'%(best_auroc))
# print('auprc %.4f'%(best_auprc))
# print('minpse %.4f'%(best_minpse))  
# logger.info('auroc %.4f'%(best_auroc))
# logger.info('auprc %.4f'%(best_auprc))
# logger.info('minpse %.4f'%(best_minpse))

In [16]:
if target_dataset == 'PD':    
    file_name = './model/pretrained-challenge-front-fill-teacher-2pd'
elif target_dataset == 'TJ':    
    file_name = './model/pretrained-challenge-front-fill-teacher-2covid'
elif target_dataset == 'HM':
    file_name = './model/pretrained-challenge-front-fill-teacher-2spain'
    
checkpoint = torch.load(file_name, \
                        map_location=torch.device(device=device))
save_epoch = checkpoint['epoch']
# print("last saved model is in epoch {}".format(save_epoch))
# logger.info("last saved model is in epoch {}".format(save_epoch))
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()

distcare_teacher(
  (PositionalEncoding): PositionalEncoding(
    (dropout): Dropout(p=0, inplace=False)
  )
  (GRUs): ModuleList(
    (0): GRU(1, 32, batch_first=True)
    (1): GRU(1, 32, batch_first=True)
    (2): GRU(1, 32, batch_first=True)
    (3): GRU(1, 32, batch_first=True)
    (4): GRU(1, 32, batch_first=True)
    (5): GRU(1, 32, batch_first=True)
    (6): GRU(1, 32, batch_first=True)
    (7): GRU(1, 32, batch_first=True)
    (8): GRU(1, 32, batch_first=True)
    (9): GRU(1, 32, batch_first=True)
    (10): GRU(1, 32, batch_first=True)
    (11): GRU(1, 32, batch_first=True)
    (12): GRU(1, 32, batch_first=True)
    (13): GRU(1, 32, batch_first=True)
    (14): GRU(1, 32, batch_first=True)
    (15): GRU(1, 32, batch_first=True)
    (16): GRU(1, 32, batch_first=True)
    (17): GRU(1, 32, batch_first=True)
    (18): GRU(1, 32, batch_first=True)
    (19): GRU(1, 32, batch_first=True)
    (20): GRU(1, 32, batch_first=True)
    (21): GRU(1, 32, batch_first=True)
    (22): GRU(1, 32, 

In [17]:
batch_loss = []
y_true = []
y_pred = []
pad_token = np.zeros(34)
with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_lens) in enumerate(batch_iter(test_x, test_y, test_x_len, batch_size, shuffle=True)):  
        optimizer.zero_grad()
        batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
        batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
        batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()

        opt, decov_loss, emb = model(batch_x, batch_lens)

        BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))
#             REC_Loss = F.mse_loss(masks * recon, masks * batch_x, reduction='mean').to(device)

        model_loss =  BCE_Loss 

        loss = model_loss
        batch_loss.append(loss.cpu().detach().numpy())
        if step % 20 == 0:
            print('Batch %d: Test Loss = %.4f'%(step, loss.cpu().detach().numpy()))
            logger.info('Batch %d: Test Loss = %.4f'%(step, loss.cpu().detach().numpy()))
        y_pred += list(opt.cpu().detach().numpy().flatten())
        y_true += list(batch_y.cpu().numpy().flatten())

print("\n==>Predicting on test")
print('Test Loss = %.4f'%(np.mean(np.array(batch_loss))))
logger.info("\n==>Predicting on test")
logger.info('Test Loss = %.4f'%(np.mean(np.array(batch_loss))))
y_pred = np.array(y_pred)
y_pred = np.stack([1 - y_pred, y_pred], axis=1)
test_res = metrics.print_metrics_binary(y_true, y_pred)

Batch 0: Test Loss = 0.0931

==>Predicting on test
Test Loss = 0.1227
confusion matrix:
[[3712   24]
 [ 133  164]]
accuracy = 0.9610711336135864
precision class 0 = 0.9654096364974976
precision class 1 = 0.8723404407501221
recall class 0 = 0.9935759902000427
recall class 1 = 0.5521885752677917
AUC of ROC = 0.9417155134499888
AUC of PRC = 0.7497692675220317
min(+P, Se) = 0.7104377104377104
f1_score = 0.6762886533793953


## Student Model

In [18]:
pad_token_source = np.zeros(34)
if target_dataset == 'PD':
    pad_token_target = np.zeros(69)
elif target_dataset == 'TJ':
    pad_token_target = np.zeros(75)
elif target_dataset == 'HM':
    pad_token_target = np.zeros(99)

In [19]:
class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

In [20]:
class distcare_student(nn.Module):
    def __init__(self, input_dim, input_diff_dim, hidden_dim, d_model,  MHD_num_head, d_ff, output_dim, keep_prob=0.5):
        super(distcare_student, self).__init__()

        # hyperparameters
        self.input_dim = input_dim  
        self.input_diff_dim = input_diff_dim
        self.hidden_dim = hidden_dim  # d_model
        self.d_model = d_model
        self.MHD_num_head = MHD_num_head
        self.d_ff = d_ff
        self.output_dim = output_dim
        self.keep_prob = keep_prob

        # layers
        self.PositionalEncoding = PositionalEncoding(self.d_model, dropout = 0, max_len = 400)
        self.GRUs = clones(nn.GRU(1, self.hidden_dim, batch_first = True), self.input_dim)
        self.generalGRUs = clones(nn.GRU(1, self.hidden_dim, batch_first = True), self.input_diff_dim)
        self.LastStepAttentions = clones(SingleAttention(self.hidden_dim, 8, attention_type='concat', demographic_dim=12, time_aware=True, use_demographic=False),self.input_dim)
        
        self.FinalAttentionQKV = FinalAttentionQKV(self.hidden_dim, self.hidden_dim, attention_type='mul',dropout = 1 - self.keep_prob)

        self.MultiHeadedAttention = MultiHeadedAttention(self.MHD_num_head, self.d_model,dropout = 1 - self.keep_prob)
        self.SublayerConnection = SublayerConnection(self.d_model, dropout = 1 - self.keep_prob)

        self.PositionwiseFeedForward = PositionwiseFeedForward(self.d_model, self.d_ff, dropout=0.1)

        self.demo_proj_main = nn.Linear(12, self.hidden_dim)
        self.demo_proj = nn.Linear(12, self.hidden_dim)
        self.Linear = nn.Linear(self.hidden_dim, 1)
        self.output = nn.Linear(self.input_dim + self.input_diff_dim, self.output_dim)

        # adversal方法中的域分类器  
        self.domain_classifier = nn.Sequential()
        self.domain_classifier.add_module('d_fc1', nn.Linear(self.hidden_dim, self.hidden_dim))
        self.domain_classifier.add_module('d_bn1', nn.BatchNorm1d(self.hidden_dim))
        self.domain_classifier.add_module('d_relu1', nn.ReLU(True))
        self.domain_classifier.add_module('d_fc2', nn.Linear(hidden_dim, 2))
        self.domain_classifier.add_module('d_softmax', nn.LogSoftmax(dim=1))

        self.dropout = nn.Dropout(p = 1 - self.keep_prob)
        self.FC_embed = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.tanh=nn.Tanh()
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.relu=nn.ReLU()
        self.to_MMD = nn.Linear(self.hidden_dim, 1)

    def forward(self, input, input_diff, lens, alpha, is_teacher):
        lens = lens.to('cpu')
        batch_size = input.size(0)
        time_step = input.size(1)
        feature_dim = input.size(2)
        feature_dim_diff = input_diff.size(2)
        assert(feature_dim == self.input_dim)# input Tensor : 256 * 48 * 76
        assert(self.d_model % self.MHD_num_head == 0)
        GRU_embeded_input = self.GRUs[0](pack_padded_sequence(input[:,:,0].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
        for i in range(feature_dim-1):
            embeded_input = self.GRUs[i+1](pack_padded_sequence(input[:,:,i+1].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
            GRU_embeded_input = torch.cat((GRU_embeded_input, embeded_input), 1)

        if is_teacher: # 来自源数据集
            General_GRU_embeded_input = self.generalGRUs[0](pack_padded_sequence(input_diff[:,:,0].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
            for i in range(feature_dim_diff - 1):
                general_embeded_input = self.generalGRUs[i + 1](pack_padded_sequence(input_diff[:,:,i].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
                General_GRU_embeded_input = torch.cat((General_GRU_embeded_input,general_embeded_input), 1)
        
            common_input = GRU_embeded_input[:, 0, :]
            for i in range(1, feature_dim):
                common_input = common_input + GRU_embeded_input[:, i, :]  
            # print(f"common_input1.shape is {common_input.shape}")
            common_input = torch.squeeze(common_input, 1) # batch * hidden
            reverse_input = ReverseLayerF.apply(common_input, alpha)
            # print(f"common_input2.shape is {common_input.shape}")
            domain_output = self.domain_classifier(reverse_input)

            posi_input = self.dropout(torch.cat((GRU_embeded_input, General_GRU_embeded_input), 1)) # batch_size * d_input + d_input_diff * hidden_dim
            
            contexts = self.Linear(posi_input).squeeze()# b i
            output = self.output(self.dropout(contexts))# b 1
            output = self.sigmoid(output)
            return output, domain_output, contexts
        else: # 来自目标数据集，主要是为了混淆domain classifier
            common_input = GRU_embeded_input[:, 0, :]
            for i in range(1, feature_dim):
                common_input = common_input + GRU_embeded_input[:, i, :]  
            common_input = torch.squeeze(common_input, 1) # batch * hidden
            reverse_input = ReverseLayerF.apply(common_input, alpha)
            domain_output = self.domain_classifier(reverse_input)
            return domain_output

        

In [21]:
def getSplitData(x, lens, y):
    train_num =int( len(x) * 0.8) + 1
    dev_num =int( len(x) * 0.1) + 1
    test_num = len(x) - train_num - dev_num
    train_x = []
    train_y = []
    train_len = []
    for idx in range(train_num):
        train_x.append(x[idx])
        train_y.append(int(y[idx][-1]))
        train_len.append(lens[idx])

    dev_x = []
    dev_y = []
    dev_len = []
    for idx in range(train_num, train_num + dev_num):
        dev_x.append(x[idx])
        dev_y.append(int(y[idx][-1]))
        dev_len.append(lens[idx])

    test_x = []
    test_y = []
    test_len = []

    for idx in range(train_num + dev_num, train_num + dev_num + test_num):
        test_x.append(x[idx])
        test_y.append(int(y[idx][-1]))
        test_len.append(lens[idx])
    return train_x, train_y, train_len, dev_x, dev_y, dev_len, test_x, test_y, test_len

In [22]:
logger.info("load target data")
if target_dataset == 'PD':
    data_path = './data/PD/'
    all_x_target = pickle.load(open(data_path + 'x.pkl', 'rb'))
    all_time_target = pickle.load(open(data_path + 'y_z.pkl', 'rb'))
    all_x_len_target = [len(i) for i in all_x_target]

    subset_idx_target = [0, 2, 3, 4, 5, 7, 8, 9, 12, 16, 17, 19, 20, 56, 57, 58]
    other_idx_target = list(range(69))
    for i in subset_idx_target:
        other_idx_target.remove(i)
    for i in range(len(all_x_target)):
        cur = np.array(all_x_target[i], dtype=float)
        cur_subset = cur[:, subset_idx_target]
        cur_other = cur[:, other_idx_target]
        all_x_target[i] = np.concatenate((cur_subset, cur_other), axis=1).tolist()
elif target_dataset == 'TJ':
    data_path = './data/Tongji/'
    all_x_target = pickle.load(open(data_path + 'x.pkl', 'rb'))
    all_y_target = pickle.load(open(data_path + 'y.pkl', 'rb'))
    all_time_target = pickle.load(open(data_path + 'y.pkl', 'rb'))
    all_x_len_target = [len(i) for i in all_x_target]

    for i in range(len(all_time_target)):
        for j in range(len(all_time_target[i])):
            all_time_target[i][j] = all_time_target[i][j][-1]
            all_y_target[i][j] = all_y_target[i][j][0]

    subset_idx_target = [2, 3, 4, 9, 13, 14, 26, 27, 30, 32, 34, 38, 39, 41, 52, 53, 66, 74]
    other_idx_target = list(range(75))
    for i in subset_idx_target:
        other_idx_target.remove(i)
    for i in range(len(all_x_target)):
        cur = np.array(all_x_target[i], dtype=float)
        cur_subset = cur[:, subset_idx_target]
        cur_other = cur[:, other_idx_target]
        all_x_target[i] = np.concatenate((cur_subset, cur_other), axis=1).tolist()
elif target_dataset == 'HM':
    data_path = './data/CDSL/'
    all_x_target = pickle.load(open(data_path + 'x.pkl', 'rb'))
    all_y_target = pickle.load(open(data_path + 'y.pkl', 'rb'))
    all_time_target = pickle.load(open(data_path + 'y.pkl', 'rb'))
    all_x_len_target = [len(i) for i in all_x_target]

    for i in range(len(all_time_target)):
        for j in range(len(all_time_target[i])):
            all_time_target[i][j] = all_time_target[i][j][-1]
            all_y_target[i][j] = all_y_target[i][j][0]

    subset_idx_target = [5, 6, 4, 2, 3, 48, 79, 76, 87, 25, 30, 31, 18, 43, 58, 66, 40, 57, 23, 92, 50, 54, 91, 60, 39, 81]
    other_idx_target= list(range(99))
    for i in subset_idx_target:
        other_idx_target.remove(i)
    for i in range(len(all_x_target)):
        cur = np.array(all_x_target[i], dtype=float)
        cur_subset = cur[:, subset_idx_target]
        cur_other = cur[:, other_idx_target]
    #     tar_all_x[i] = np.concatenate((cur_subset, cur_other), axis=1).tolist()
        all_x_target[i] = np.concatenate((cur_subset, cur_other), axis=1).tolist()
    
if target_dataset == 'PD':
    all_x_target = all_x_target
    all_y_target = all_time_target
elif  target_dataset == 'HM' or target_dataset == 'TJ':
    examples = []
    for idx in range(len(all_x_target)):
        examples.append((all_x_target[idx], all_y_target[idx], all_time_target[idx], all_x_len_target[idx]))
    examples = sorted(examples, key=lambda e: len(e[0]), reverse=True)
    all_x_target = [e[0] for e in examples]
    all_y_target = [e[1] for e in examples]
    all_time_target = [e[2] for e in examples]
    all_x_len_target = [e[3] for e in examples]

num_source = len(all_x_source)
num_target = len(all_x_target)
# print(target_dataset,len(all_x_target), len(all_x_target[0]),len(all_x_target[0][0]))
all_x_target_confuse = []
all_x_len_target_confuse = []
all_y_target_confuse = []
all_x_source_confuse = []
all_x_len_source_confuse = []
all_y_source_confuse = []
repeat_times = 0

if num_source < num_target:
    all_x_target_confuse = all_x_target
    all_y_target_confuse = all_y_target
    all_x_len_target_confuse = all_x_len_target
    while repeat_times * num_source < num_target:
        all_x_source_confuse = all_x_source_confuse + all_x_source
        all_x_len_source_confuse = all_x_len_source_confuse + all_x_len_source
        all_y_source_confuse =  all_y_source_confuse + all_y_source
        repeat_times = repeat_times + 1
    all_x_source_confuse = all_x_source_confuse[:num_target]
    all_x_len_source_confuse = all_x_len_source_confuse[:num_target]
    all_y_source_confuse = all_y_source_confuse[:num_target]
elif num_target < num_source:
    all_x_source_confuse = all_x_source
    all_x_len_source_confuse = all_x_len_source
    all_y_source_confuse = all_y_source
    while repeat_times * num_target < num_source:
        all_x_target_confuse = all_x_target_confuse + all_x_target
        all_x_len_target_confuse = all_x_len_target_confuse + all_x_len_target
        all_y_target_confuse = all_y_target_confuse + all_y_target
        repeat_times = repeat_times + 1
    all_x_target_confuse = all_x_target_confuse[:num_source]
    all_x_len_target_confuse = all_x_len_target_confuse[:num_source]
    all_y_target_confuse = all_y_target_confuse[:num_source]

# print(f"len(all_x_source_confuse) is {len(all_x_source_confuse)}, len(all_x_target_confuse) is {len(all_x_target_confuse)}")

#todo 划分train、dev、test 
# all_x_source_confuse = pad_sents(all_x_source_confuse, pad_token_source)
# all_x_target_confuse = pad_sents(all_x_target_confuse, pad_token_target)
train_x_source_confuse, train_y_source_confuse, train_len_source_confuse, dev_x_source_confuse, dev_y_source_confuse, dev_len_source_confuse, test_x_source_confuse,\
test_y_source_confuse, test_len_source_confuse = getSplitData(all_x_source_confuse, all_x_len_source_confuse, all_y_source_confuse)

train_x_target_confuse, train_y_target_confuse, train_len_target_confuse, dev_x_target_confuse, dev_y_target_confuse, dev_len_target_confuse, test_x_target_confuse,\
test_y_target_confuse, test_len_target_confuse = getSplitData(all_x_target_confuse, all_x_len_target_confuse, all_y_target_confuse)

# long_x_source = all_x_source
# long_y_source = [y[-1] for y in all_y_source]



In [23]:
epochs = 50
batch_size = 256
common_dim = subset_cnt 

diff_dim = input_dim - subset_cnt
hidden_dim = 64
d_model = 64
MHD_num_head = 4
d_ff = 64
output_dim = 1
model_student = distcare_student(input_dim = common_dim, input_diff_dim = diff_dim, hidden_dim = hidden_dim, d_model=d_model, MHD_num_head=MHD_num_head, d_ff=d_ff, output_dim = output_dim).to(device)
optimizer_student = torch.optim.Adam(model_student.parameters(), lr=1e-3)

## Training student model

In [24]:
class MultitaskLoss(nn.Module):
    def __init__(self, task_num=2):
        super(MultitaskLoss, self).__init__()
        self.task_num = task_num
        self.alpha = nn.Parameter(torch.ones((task_num)), requires_grad=True)
        self.bce = nn.BCELoss()
        self.kl = nn.KLDivLoss(reduce=True, size_average=True)

    def forward(self, opt_student, batch_y, emb_student, emb_teacher, tar_source, tar_tar):
        BCE_Loss = self.bce(opt_student, batch_y)
        emb_Loss = self.kl(emb_student, emb_teacher)
        return BCE_Loss * self.alpha[0] + emb_Loss * self.alpha[1]

def get_multitask_loss(opt_student, batch_y, emb_student, emb_teacher):
    mtl = MultitaskLoss(task_num=3)
    return mtl(opt_student, batch_y, emb_student, emb_teacher)

In [25]:
#Generate Teacher model embedding
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()

train_teacher_emb = []
batch_loss = []
y_true = []
y_pred = []
pad_token = np.zeros(34)
with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_lens) in enumerate(batch_iter(train_x_source_confuse, train_y_source_confuse, train_len_source_confuse, batch_size, shuffle=False)):  
        optimizer.zero_grad()
        batch_x = torch.tensor(pad_sents(batch_x, pad_token_source), dtype=torch.float32).to(device)
        batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
        batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()

        masks = length_to_mask(batch_lens).unsqueeze(-1).float()

        opt, decov_loss, emb = model(batch_x, batch_lens)
        train_teacher_emb.append(emb.cpu().detach().numpy())

        BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))
#             REC_Loss = F.mse_loss(masks * recon, masks * batch_x, reduction='mean').to(device)

        model_loss =  BCE_Loss 
        if step % 20 == 0:
            print('Batch %d: Test Loss = %.4f'%(step, model_loss.cpu().detach().numpy()))
            logger.info('Batch %d: Test Loss = %.4f'%(step, model_loss.cpu().detach().numpy()))

Batch 0: Test Loss = 0.1131
Batch 20: Test Loss = 0.1478
Batch 40: Test Loss = 0.1526
Batch 60: Test Loss = 0.1207
Batch 80: Test Loss = 0.0851
Batch 100: Test Loss = 0.0724
Batch 120: Test Loss = 0.1097


In [26]:
# class MyDataset(Dataset):
#     def __init__(self, data, len, labels):
#         self.data = data
#         self.len = len
#         self.labels = labels

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         sample = self.data[idx]
#         len = self.len[idx]
#         label = self.labels[idx]
#         return sample, len, label

# batch_size = 256
# train_source_dataset = MyDataset(train_x_source_confuse, train_len_source_confuse, train_y_source_confuse)
# train_target_dataset = MyDataset(train_x_target_confuse, train_len_target_confuse, train_y_target_confuse)
# train_source_dataloader = DataLoader(train_source_dataset, batch_size= batch_size)
# train_target_dataloader = DataLoader(train_target_dataset, batch_size=batch_size)
# train_source_data_iter = iter(train_source_dataloader)
# train_target_data_iter = iter(train_target_dataloader)

In [27]:
# # Training Student
# # If you don't want to train Student Model:
# # - The pretrained student model is in direcrtory './model/', and can be directly loaded, 
# # - Simply skip this cell and load the model to validate on Dev Dataset.

# logger.info('Training Student')
# teacher_flag = True
# epochs = 30
# total_train_loss = []
# total_valid_loss = []
# global_best = 0
# auroc = []
# auprc = []
# minpse = []
# history = []
# # begin_time = time.time()
# best_auroc = 0
# best_auprc = 0
# best_minpse = 0
# best_total_loss = 0x3f3f3f3f
# loss_domain = torch.nn.NLLLoss()
# loss_predict = torch.nn.MSELoss()
# loss_embed = nn.KLDivLoss(reduce=True, size_average=True)




# print(f'len(train_source_iter) is {len(train_x_source_confuse)}, len(train_target_iter) is {len(train_x_target_confuse)}, steps is {len(train_x_source_confuse) // batch_size + 1}')

# if target_dataset == 'PD':
#     data_str = 'pd'
# elif target_dataset == 'TJ':
#     data_str = 'covid'
# elif target_dataset == 'HM':
#     data_str = 'spain'


# if teacher_flag:
#     file_name = './model/pretrained-challenge-front-fill-2'+ data_str
# else: 
#     file_name = './model/pretrained-challenge-front-fill-2'+ data_str + '-noteacher'

# for each_epoch in range(epochs):
#     train_source_iter = batch_iter(train_x_source_confuse, train_y_source_confuse, train_len_source_confuse, batch_size=batch_size)
#     dev_source_iter = batch_iter(dev_x_source_confuse, dev_y_source_confuse, dev_len_source_confuse, batch_size=batch_size)
#     test_source_iter = batch_iter(test_x_source_confuse, test_y_source_confuse, test_len_source_confuse, batch_size=batch_size)
#     train_target_iter = batch_iter(train_x_target_confuse, train_y_target_confuse, train_len_target_confuse, batch_size=batch_size)
#     dev_target_iter = batch_iter(dev_x_target_confuse, dev_y_target_confuse, dev_len_target_confuse, batch_size=batch_size)
#     test_target_iter = batch_iter(test_x_target_confuse, test_y_target_confuse, test_len_target_confuse, batch_size=batch_size)
#     epoch_loss = []
#     counter_batch = 0
#     model_student.train()  
#     model.eval()
#     steps = len(train_x_source_confuse) // batch_size + 1 if len(train_x_source_confuse) % batch_size != 0 else len(train_x_source_confuse) // batch_size
#     for step in range(steps):
#         # -----source_domain--------
#         batch_x, batch_y, batch_lens= next(train_source_iter)
#         p = float(step + each_epoch * steps) / epochs / steps
#         alpha = 2. / (1. + np.exp(-10 * p)) - 1
#         optimizer_student.zero_grad()
#         batch_x = torch.tensor(pad_sents(batch_x, pad_token_source), dtype=torch.float32).to(device)
#         batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
#         batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
#         # batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)
#         # opt_student, decov_loss_student, emb_student, tar_result = model_student(batch_x[:,:,:subset_cnt], batch_x[:,:,subset_cnt:], batch_lens, [tar_all_x, tar_all_x_len], True)
#         domain_label = torch.zeros(min(batch_size, batch_x.shape[0])).long().to(device)
#         opt_student, opt_domain, emb_student = model_student(batch_x[:,:,:subset_cnt], batch_x[:,:,subset_cnt:], batch_lens, alpha, True)
#         emb_teacher = torch.tensor(train_teacher_emb[step], dtype=torch.float32).to(device)
#         emb_student = F.log_softmax(emb_student, dim=1)
#         emb_teacher = F.softmax(emb_teacher.detach(), dim=1)
#         err_emb = loss_embed(emb_student, emb_teacher)
#         err_predict = loss_predict(opt_student, batch_y)
#         err_domain1 = loss_domain(opt_domain, domain_label)
#             # loss = get_multitask_loss(opt_student, batch_y.unsqueeze(-1), emb_student, emb_teacher)

#         # -----target_domain--------
#         batch_x, batch_y, batch_lens = next(train_target_iter)
#         p = float(step + each_epoch * len(train_x_source)) / epochs / len(train_x_len_source)
#         alpha = 2. / (1. + np.exp(-10 * p)) - 1
#         optimizer_student.zero_grad()
#         batch_x = torch.tensor(pad_sents(batch_x, pad_token_target), dtype=torch.float32).to(device)
#         batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
#         batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
#         # batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)
#         # opt_student, decov_loss_student, emb_student, tar_result = model_student(batch_x[:,:,:subset_cnt], batch_x[:,:,subset_cnt:], batch_lens, [tar_all_x, tar_all_x_len], True)
#         domain_label = torch.ones(min(batch_size, batch_x.shape[0])).long().to(device)
#         opt_domain = model_student(batch_x[:,:,:subset_cnt], batch_x[:,:,subset_cnt:], batch_lens, alpha, False)
#         err_domain2 = loss_domain(opt_domain, domain_label)

#         # -----common--------
#         loss = err_emb + err_predict + err_domain1 + err_domain2
#         epoch_loss.append(loss.cpu().detach().numpy())
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model_student.parameters(), 20)
#         optimizer_student.step()

#         if step % 20 == 0:
#             print('Epoch %d Batch %d: Train Loss = %.4f'%(each_epoch, step, loss.cpu().detach().numpy()))
#             logger.info('Epoch %d Batch %d: Train Loss = %.4f'%(each_epoch, step, loss.cpu().detach().numpy()))

#     epoch_loss = np.mean(epoch_loss)
#     total_train_loss.append(epoch_loss)


#     # dev_source_dataset = MyDataset(dev_x_source_confuse, dev_len_source_confuse, dev_y_source_confuse)
#     # dev_target_dataset = MyDataset(dev_x_target_confuse, dev_len_target_confuse, dev_y_target_confuse)
#     # dev_source_dataloader = DataLoader(dev_source_dataset, batch_size= batch_size)
#     # dev_target_dataloader = DataLoader(dev_target_dataset, batch_size=batch_size)
#     #Validation

#     y_true = []
#     y_pred = []
#     with torch.no_grad():
#         steps = len(dev_x_source_confuse) // batch_size + 1 if len(dev_x_source_confuse) % batch_size != 0 else len(dev_x_source_confuse) // batch_size
#         for step in range(steps):
#             # -----source_domain--------
#             batch_x, batch_y, batch_lens= next(dev_source_iter)
#             p = float(step + each_epoch * steps) / epochs / steps
#             alpha = 2. / (1. + np.exp(-10 * p)) - 1
#             optimizer_student.zero_grad()
#             batch_x = torch.tensor(pad_sents(batch_x, pad_token_source), dtype=torch.float32).to(device)
#             batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
#             batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
#             # batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)
#             # opt_student, decov_loss_student, emb_student, tar_result = model_student(batch_x[:,:,:subset_cnt], batch_x[:,:,subset_cnt:], batch_lens, [tar_all_x, tar_all_x_len], True)
#             domain_label = torch.zeros(min(batch_size, batch_x.shape[0])).long().to(device)
#             opt_student, opt_domain, emb_student = model_student(batch_x[:,:,:subset_cnt], batch_x[:,:,subset_cnt:], batch_lens, alpha, True)
#             # emb_teacher = torch.tensor(dev_teacher_emb[step], dtype=torch.float32).to(device)
#             emb_student = F.log_softmax(emb_student, dim=1)
#             emb_teacher = F.softmax(emb_teacher.detach(), dim=1)
#             # err_emb = loss_embed(emb_student, emb_teacher) #todo 是否考虑它
#             err_predict = loss_predict(opt_student, batch_y)
#             err_domain1 = loss_domain(opt_domain, domain_label)
#                 # loss = get_multitask_loss(opt_student, batch_y.unsqueeze(-1), emb_student, emb_teacher)

#             # -----target_domain--------
#             batch_x, batch_y, batch_lens = next(dev_target_iter)
#             optimizer_student.zero_grad()
#             batch_x = torch.tensor(pad_sents(batch_x, pad_token_target), dtype=torch.float32).to(device)
#             batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
#             batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
#             # batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)
#             # opt_student, decov_loss_student, emb_student, tar_result = model_student(batch_x[:,:,:subset_cnt], batch_x[:,:,subset_cnt:], batch_lens, [tar_all_x, tar_all_x_len], True)
#             domain_label = torch.ones(min(batch_size, batch_x.shape[0])).long().to(device)
#             opt_domain = model_student(batch_x[:,:,:subset_cnt], batch_x[:,:,subset_cnt:], batch_lens, alpha, False)
#             err_domain2 = loss_domain(opt_domain, domain_label)

#             # -----common--------
#             loss = err_predict + err_domain1 + err_domain2
#             if loss < best_total_loss:
#                 best_total_loss = loss
#                 state = {
#                     'net': model_student.state_dict(),
#                     'optimizer': optimizer_student.state_dict(),
#                     'epoch': each_epoch
#                 }
#                 torch.save(state, file_name)
#                 print('------------ Save best model - TOTAL_LOSS: %.4f ------------'%best_total_loss)
#                 logger.info('------------ Save best model - TOTAL_LOSS: %.4f ------------'%best_total_loss)




In [1]:
teacher_flag = True

if target_dataset == 'PD':    
    data_str = 'pd'
elif target_dataset == 'TJ':    
    data_str = 'covid'
elif target_dataset == 'HM':
    data_str = 'spain'

if teacher_flag:
    file_name = './model/pretrained-challenge-front-fill-2'+ data_str
else: 
    file_name = './model/pretrained-challenge-front-fill-2'+ data_str + '-noteacher'

checkpoint = torch.load(file_name, \
                        map_location=torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu') )
save_epoch = checkpoint['epoch']
print("last saved model is in epoch {}".format(save_epoch))
logger.info("last saved model is in epoch {}".format(save_epoch))
model_student.load_state_dict(checkpoint['net'])
optimizer_student.load_state_dict(checkpoint['optimizer'])
model_student.eval()

NameError: name 'target_dataset' is not defined

# evaluate student model

In [ ]:
#anchor
batch_loss = []
y_true = []
y_pred = []
with torch.no_grad():
    model_student.eval()
    # print(f"all_x_len_source_confuse.length is {len(all_x_len_source_confuse)}, all_x_source_confuse is {len(all_x_source_confuse)}")
    test_source_iter = batch_iter(test_x_source_confuse, test_y_source_confuse, test_len_source_confuse, batch_size=batch_size, shuffle=True)
    steps = len(test_x_source_confuse) // batch_size + 1 if len(test_x_source_confuse) % batch_size != 0 else len(test_x_source_confuse) // batch_size
    for step in range(steps):
        # -----source_domain--------
        batch_x, batch_y, batch_lens= next(test_source_iter) 
        optimizer_student.zero_grad()
        batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
        batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
        batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
        masks = length_to_mask(batch_lens).unsqueeze(-1).float()
        opt, _, _  = model_student(batch_x[:,:,:subset_cnt], batch_x[:,:,subset_cnt:], batch_lens, 1, True)

        BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))
#             REC_Loss = F.mse_loss(masks * recon, masks * batch_x, reduction='mean').to(device)

        model_loss =  BCE_Loss 

        loss = model_loss
        batch_loss.append(loss.cpu().detach().numpy())
        if step % 20 == 0:
            print('Batch %d: Test Loss = %.4f'%(step, loss.cpu().detach().numpy()))
            logger.info('Batch %d: Test Loss = %.4f'%(step, loss.cpu().detach().numpy()))
        y_pred += list(opt.cpu().detach().numpy().flatten())
        y_true += list(batch_y.cpu().numpy().flatten())

print("\n==>Predicting on test")
print('Test Loss = %.4f'%(np.mean(np.array(batch_loss))))
logger.info("\n==>Predicting on test")
logger.info('Test Loss = %.4f'%(np.mean(np.array(batch_loss))))
y_pred = np.array(y_pred)
y_pred = np.stack([1 - y_pred, y_pred], axis=1)
test_res = metrics.print_metrics_binary(y_true, y_pred)

## Transfer Target Dataset & Model

In [ ]:
if target_dataset == 'PD':
    source_common_idx = [31, 29, 28, 33, 25, 18, 7, 21, 16, 15, 19, 17, 24, 3, 5, 0]
    target_common_idx = [0, 2, 3, 4, 5, 7, 8, 9, 12, 16, 17, 19, 20, 56, 57, 58]
    source_data_path = './data/Challenge/'
    source_x = pickle.load(open(source_data_path + 'new_x_front_fill.dat', 'rb'))
    target_data_path = './data/PD/'
    target_x = pickle.load(open(target_data_path + 'x.pkl', 'rb'))
elif target_dataset == 'TJ':
    source_common_idx = [27, 29, 18, 16, 26, 33, 28, 31, 32, 15, 11, 25, 21, 20, 9, 17, 30, 19]
    target_common_idx = [2, 3, 4, 9, 13, 14, 26, 27, 30, 32, 34, 38, 39, 41, 52, 53, 66, 74]
    source_data_path = './data/Challenge/'
    source_x = pickle.load(open(source_data_path + 'new_x_front_fill.dat', 'rb'))
    target_data_path = './data/Tongji/'
    target_x = pickle.load(open(target_data_path + 'x.pkl', 'rb'))

elif target_dataset == 'HM':
    source_common_idx = [0, 1, 2, 3, 5, 9, 11, 12, 13, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
    target_common_idx = [5, 6, 4, 2, 3, 48, 79, 76, 87, 25, 30, 31, 18, 43, 58, 66, 40, 57, 23, 92, 50, 54, 91, 60, 39, 81]
    source_data_path = './data/Challenge/'
    source_x = pickle.load(open(source_data_path + 'new_x_front_fill.dat', 'rb'))
    target_data_path = './data/CDSL/'
    target_x = pickle.load(open(target_data_path + 'x.pkl', 'rb'))

assert(len(source_common_idx) == len(target_common_idx))
common_len = len(source_common_idx)
source_x_diff = []
target_x_diff = []

source_total_len = 34
source_other_idx = list(range(source_total_len))
for i in source_common_idx:
    source_other_idx.remove(i)

if target_dataset == 'PD':
    target_total_len = 69
    target_other_idx = list(range(target_total_len))
    for i in target_common_idx:
        target_other_idx.remove(i)
elif target_dataset == 'TJ':
    target_other_idx = list(range(75))
    target_total_len = 75
    for i in target_common_idx:
        target_other_idx.remove(i)
elif target_dataset == 'HM':
    target_other_idx = list(range(99))
    target_total_len = 99
    for i in target_common_idx:
        target_other_idx.remove(i)

for i in range(len(source_x)):
    cur = np.array(source_x[i], dtype=float)
    cur_subset = cur[:, source_common_idx]
    cur_other = cur[:, source_other_idx]
    source_x_diff.append(cur_other.tolist())
    source_x[i] = np.concatenate((cur_subset, cur_other), axis=1).tolist()

for i in range(len(target_x)):
    cur = np.array(target_x[i], dtype=float)
    cur_subset = cur[:, target_common_idx]
    cur_other = cur[:, target_other_idx]
    target_x_diff.append(cur_other.tolist())
    target_x[i] = np.concatenate((cur_subset, cur_other), axis=1).tolist()


source_max = 0
for i in range(len(source_x_diff)):
    if source_max < len(source_x_diff[i]):
        source_max = len(source_x_diff[i])

source_x_diff_longest = max(list(len(_) for _ in source_x_diff))
source_x_longest = max(list(len(_) for _ in source_x))
source_batch = len(source_x_diff)
source_diff_features = source_total_len - common_len
source_x_diff_ex = torch.zeros((source_batch, source_x_diff_longest, source_diff_features))
source_x_ex = torch.zeros((source_batch, source_x_longest, source_total_len))

for i in range(len(source_x_diff)):
    for j in range(source_x_diff_longest):
        cur_len = len(source_x_diff[i])
        if j < cur_len:
            source_x_diff_ex[i,j,:] = torch.Tensor(source_x_diff[i])[j,:]
        else:
            source_x_diff_ex[i,j,:] = torch.Tensor(source_x_diff[i])[cur_len - 1,:]

for i in range(len(source_x)):
    for j in range(source_x_longest):
        cur_len = len(source_x[i])
        if j < cur_len:
            source_x_ex[i,j,:] = torch.Tensor(source_x[i])[j,:]
        else:
            source_x_ex[i,j,:] = torch.Tensor(source_x[i])[cur_len - 1,:]

target_x_diff_longest = max(list(len(_) for _ in target_x_diff))
target_batch = len(target_x_diff)
target_features = target_total_len - common_len
target_x_diff_ex = torch.zeros((target_batch, target_x_diff_longest, target_features))

for i in range(len(target_x_diff)):
    for j in range(target_x_diff_longest):
        cur_len = len(target_x_diff[i])
        if j < cur_len:
            target_x_diff_ex[i,j,:] = torch.Tensor(target_x_diff[i])[j,:]
        else:
            target_x_diff_ex[i,j,:] = torch.Tensor(target_x_diff[i])[cur_len - 1,:]





In [ ]:
print(f'source_x_diff_ex.shape is {np.array(source_x_diff_ex).shape}, max_len is {source_max}')
print(f'source_x_ex.shape is {np.array(source_x_ex.shape)}')
print(f'target_x_diff_ex.shape is {np.array(target_x_diff_ex).shape}')

In [ ]:
source_x_diff_mean = np.mean(np.array(source_x_diff_ex), 0)
source_x_mean = np.mean(np.array(source_x_ex), 0 )
target_x_diff_mean = np.mean(np.array(target_x_diff_ex), 0)
source_x_diff_mean.shape, target_x_diff_mean.shape

In [ ]:
from dtw import *
latest_idx = []
for i in range(target_x_diff_mean.shape[1]):
    min_idx = 0
    min_distance = float('inf')
    for j in range(source_x_mean.shape[1]):
        source_feature = source_x_mean[:, j]
        target_feature = target_x_diff_mean[:, i]
        alignment = dtw(source_feature, target_feature)
        distance = alignment.distance
        if min_distance > distance:
            min_distance = distance
            min_idx = j
    latest_idx.append(min_idx)
latest_idx

In [ ]:
logger.info("Transfer Target Dataset & Model")

if target_dataset == 'PD':
    data_path = './data/PD/'
    all_x = pickle.load(open(data_path + 'x.pkl', 'rb'))
    all_time = pickle.load(open(data_path + 'y_z.pkl', 'rb'))
    all_x_len = [len(i) for i in all_x]

    tar_subset_idx = [0, 2, 3, 4, 5, 7, 8, 9, 12, 16, 17, 19, 20, 56, 57, 58]
    tar_other_idx = list(range(69))
    for i in tar_subset_idx:
        tar_other_idx.remove(i)
    for i in range(len(all_x)):
        cur = np.array(all_x[i], dtype=float)
        cur_subset = cur[:, tar_subset_idx]
        cur_other = cur[:, tar_other_idx]
        all_x[i] = np.concatenate((cur_subset, cur_other), axis=1).tolist()
elif target_dataset == 'TJ':
    data_path = './data/Tongji/'
    all_x = pickle.load(open(data_path + 'x.pkl', 'rb'))
    all_y = pickle.load(open(data_path + 'y.pkl', 'rb'))
    all_time = pickle.load(open(data_path + 'y.pkl', 'rb'))
    all_x_len = [len(i) for i in all_x]

    for i in range(len(all_time)):
        for j in range(len(all_time[i])):
            all_time[i][j] = all_time[i][j][-1]
            all_y[i][j] = all_y[i][j][0]

    tar_subset_idx = [2, 3, 4, 9, 13, 14, 26, 27, 30, 32, 34, 38, 39, 41, 52, 53, 66, 74]
    tar_other_idx = list(range(75))
    for i in tar_subset_idx:
        tar_other_idx.remove(i)
    for i in range(len(all_x)):
        cur = np.array(all_x[i], dtype=float)
        cur_subset = cur[:, tar_subset_idx]
        cur_other = cur[:, tar_other_idx]
        all_x[i] = np.concatenate((cur_subset, cur_other), axis=1).tolist()
elif target_dataset == 'HM':
    data_path = './data/CDSL/'
    all_x = pickle.load(open(data_path + 'x.pkl', 'rb'))
    all_y = pickle.load(open(data_path + 'y.pkl', 'rb'))
    all_time = pickle.load(open(data_path + 'y.pkl', 'rb'))
    all_x_len = [len(i) for i in all_x]

    for i in range(len(all_time)):
        for j in range(len(all_time[i])):
            all_time[i][j] = all_time[i][j][-1]
            all_y[i][j] = all_y[i][j][0]

    tar_subset_idx = [5, 6, 4, 2, 3, 48, 79, 76, 87, 25, 30, 31, 18, 43, 58, 66, 40, 57, 23, 92, 50, 54, 91, 60, 39, 81]
    tar_other_idx = list(range(99))
    for i in tar_subset_idx:
        tar_other_idx.remove(i)
    for i in range(len(all_x)):
        cur = np.array(all_x[i], dtype=float)
        cur_subset = cur[:, tar_subset_idx]
        cur_other = cur[:, tar_other_idx]
        all_x[i] = np.concatenate((cur_subset, cur_other), axis=1).tolist()
    
print(all_x[0])
print(len(all_x[0][0]))
print(len(all_x))
logger.info(all_x[0])
logger.info(len(all_x[0][0]))
logger.info(len(all_x))

In [ ]:
long_x = all_x
# long_y = all_y
# long_y_kfold = [each[-1] for each in all_y]
long_time = all_time

In [ ]:

def get_n2n_data(x, y, x_len):
    length = len(x)
    assert length == len(y)
    assert length == len(x_len)
    new_x = []
    new_y = []
    new_x_len = []
    for i in range(length):
        for j in range(len(x[i])):
            new_x.append(x[i][:j+1])
            new_y.append(y[i][j])
            new_x_len.append(j+1)
    return new_x, new_y, new_x_len

In [ ]:
class distcare_target(nn.Module):
    def __init__(self, input_dim, hidden_dim, d_model,  MHD_num_head, d_ff, output_dim, keep_prob=0.5):
        super(distcare_target, self).__init__()

        # hyperparameters
        self.input_dim = input_dim  
        self.hidden_dim = hidden_dim  # d_model
        self.d_model = d_model
        self.MHD_num_head = MHD_num_head
        self.d_ff = d_ff
        self.output_dim = output_dim
        self.keep_prob = keep_prob

        # layers
        self.PositionalEncoding = PositionalEncoding(self.d_model, dropout = 0, max_len = 400)

        self.GRUs = clones(nn.GRU(1, self.hidden_dim, batch_first = True), self.input_dim)
        
        self.LastStepAttentions = clones(SingleAttention(self.hidden_dim, 16, attention_type='concat', demographic_dim=12, time_aware=True, use_demographic=False),self.input_dim)
        
        self.FinalAttentionQKV = FinalAttentionQKV(self.hidden_dim, self.hidden_dim, attention_type='mul',dropout = 1 - self.keep_prob)

        self.MultiHeadedAttention = MultiHeadedAttention(self.MHD_num_head, self.d_model,dropout = 1 - self.keep_prob)
        self.SublayerConnection = SublayerConnection(self.d_model, dropout = 1 - self.keep_prob)

        self.PositionwiseFeedForward = PositionwiseFeedForward(self.d_model, self.d_ff, dropout=0.1)

        self.demo_proj_main = nn.Linear(12, self.hidden_dim)
        self.demo_proj = nn.Linear(12, self.hidden_dim)
        self.output = nn.Linear(self.hidden_dim, self.output_dim)

        self.dropout = nn.Dropout(p = 1 - self.keep_prob)
        self.FC_embed = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.tanh=nn.Tanh()
        self.Linear = nn.Linear(self.hidden_dim, 1)
        self.Linear_los = nn.Linear(self.input_dim, self.output_dim)
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.relu=nn.ReLU()

    def forward(self, input, lens):
        lens = lens.to('cpu')
        # input shape [batch_size, timestep, feature_dim]
#         demo_main = self.tanh(self.demo_proj_main(demo_input)).unsqueeze(1)# b hidden_dim
        
        batch_size = input.size(0)
        time_step = input.size(1)
        feature_dim = input.size(2)
        assert(feature_dim == self.input_dim)# input Tensor : 256 * 48 * 76
        assert(self.d_model % self.MHD_num_head == 0)

        # Initialization
        #cur_hs = Variable(torch.zeros(batch_size, self.hidden_dim).unsqueeze(0))

        # forward
        # GRU_embeded_input = self.GRUs[0](input[:,:,0].unsqueeze(-1), Variable(torch.zeros(batch_size, self.hidden_dim).unsqueeze(0)).to(device))[0] # b t h
        # Attention_embeded_input = self.LastStepAttentions[0](GRU_embeded_input)[0].unsqueeze(1)# b 1 h
        # for i in range(feature_dim-1):
        #     embeded_input = self.GRUs[i+1](input[:,:,i+1].unsqueeze(-1), Variable(torch.zeros(batch_size, self.hidden_dim).unsqueeze(0)).to(device))[0] # b 1 h
        #     embeded_input = self.LastStepAttentions[i+1](embeded_input)[0].unsqueeze(1)# b 1 h
        #     Attention_embeded_input = torch.cat((Attention_embeded_input, embeded_input), 1)# b i h

        # Attention_embeded_input = torch.cat((Attention_embeded_input, demo_main), 1)# b i+1 h
        # posi_input = self.dropout(Attention_embeded_input) # batch_size * d_input+1 * hidden_dim

#         input = pack_padded_sequence(input, lens, batch_first=True)
        
        GRU_embeded_input = self.GRUs[0](pack_padded_sequence(input[:,:,0].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
#         print(GRU_embeded_input.shape)
        for i in range(feature_dim-1):
            embeded_input = self.GRUs[i+1](pack_padded_sequence(input[:,:,i+1].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
            GRU_embeded_input = torch.cat((GRU_embeded_input, embeded_input), 1)
        

#         GRU_embeded_input = torch.cat((GRU_embeded_input, demo_main), 1)# b i+1 h
        posi_input = self.dropout(GRU_embeded_input) # batch_size * d_input * hidden_dim
        contexts = self.Linear(posi_input).squeeze()# b i
        output = self.Linear_los(self.dropout(contexts))# b 1
        #mask = subsequent_mask(time_step).to(device) # 1 t t 下三角 N to 1任务不用mask
        return output, None, None
    #, self.MultiHeadedAttention.attn


In [ ]:
def transfer_gru_dict(pretrain_dict, model_dict, latest_idx, common_len):
    state_dict = {}
    
    for k, v in model_dict.items():
        model_point_position1 = k.find('.')
        model_module_name = k[:model_point_position1]
        if "GRUs" == model_module_name:
            model_point_position2 = k.find('.', model_point_position1+1)
            model_module_idx = int(k[model_point_position1 + 1: model_point_position2])
            print(f'model_module_idx is {model_module_idx}')
            if model_module_idx < common_len:
                state_dict[k] = pretrain_dict[k]
            # else:
            #     diff_idx = model_module_idx - common_len
            #     target_module_idx = int(str(latest_idx[diff_idx]))
            #     if target_module_idx < common_len:
            #         target_module_name = "GRUs"
            #         target_k = target_module_name +'.' + str(target_module_idx) + '.' + k[model_point_position2+1:]
            #         state_dict[k] = pretrain_dict[target_k]
            #     else:
            #         target_module_name = "generalGRUs"
            #         target_k = target_module_name +'.' + str(target_module_idx - common_len) + '.' + k[model_point_position2+1:]
            #         state_dict[k] = pretrain_dict[target_k]
    return state_dict

In [ ]:
if target_dataset == 'PD':
    input_dim = 69
elif target_dataset == 'TJ':
    input_dim = 75
elif target_dataset == 'HM':
    input_dim = 99
    
cell = 'GRU'
hidden_dim = 64
d_model = 64
MHD_num_head = 4
d_ff = 64
output_dim = 1

In [ ]:
def ckd_batch_iter(x, y, lens, batch_size, shuffle=False):
    """ Yield batches of source and target sentences reverse sorted by length (largest to smallest).
    @param data (list of (src_sent, tgt_sent)): list of tuples containing source and target sentence
    @param batch_size (int): batch size
    @param shuffle (boolean): whether to randomly shuffle the dataset
    """
    batch_num = math.ceil(len(x) / batch_size) # 向下取整
    index_array = list(range(len(x)))

    if shuffle:
        np.random.shuffle(index_array)

    for i in range(batch_num):
        indices = index_array[i * batch_size: (i + 1) * batch_size] #  fetch out all the induces
        
        examples = []
        for idx in indices:
            examples.append((x[idx], y[idx],  lens[idx]))
       
        examples = sorted(examples, key=lambda e: len(e[0]), reverse=True)
    
        batch_x = [e[0] for e in examples]
        batch_y = [e[1] for e in examples]
#         batch_name = [e[2] for e in examples]
        batch_lens = [e[2] for e in examples]
       

        yield batch_x, batch_y, batch_lens

In [ ]:
class TargetMultitaskLoss(nn.Module):
    def __init__(self, task_num=2):
        super(TargetMultitaskLoss, self).__init__()
        self.task_num = task_num
        self.alpha = nn.Parameter(torch.ones((task_num)), requires_grad=True)
        self.mse = nn.MSELoss()
        self.bce = nn.BCELoss()

    def forward(self, opt_student, los, outcome, outcome_y):
        MSE_Loss = self.mse(opt_student, los)
        BCE_Loss = self.bce(outcome, outcome_y)
        return MSE_Loss * self.alpha[0] + BCE_Loss * self.alpha[1]

def get_target_multitask_loss(opt_student, los, outcome, outcome_y):
    mtl = TargetMultitaskLoss(task_num=2)
    return mtl(opt_student, los, outcome, outcome_y)

def reverse_los(y, los_info):
    return y * los_info["los_std"] + los_info["los_mean"]

In [ ]:
los_info = pickle.load(open(data_path + 'los_info.pkl', 'rb'))
print(los_info)
logger.info(los_info)

In [ ]:
if target_dataset == 'PD':
    n_splits = 5
    epochs = 30
elif target_dataset == 'TJ':
    n_splits = 10
    epochs = 150
elif target_dataset == 'HM':
    n_splits = 3
    epochs = 20

teacher_flag = True
transfer_flag = True
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

if target_dataset == 'PD':    
    data_str = 'pd'
elif target_dataset == 'TJ':    
    data_str = 'covid'
elif target_dataset == 'HM':
    data_str = 'spain'

if teacher_flag:
    file_name = './model/pretrained-challenge-front-fill-2'+ data_str
else: 
    file_name = './model/pretrained-challenge-front-fill-2'+ data_str + '-noteacher'


batch_size = 256

fold_count = 0
total_train_loss = []
total_valid_loss = []

global_best = 10000
mse = []
mad = []
mape = []
kappa = []
history = []

pad_token = np.zeros(input_dim)
# begin_time = time.time()

for train, test in kfold.split(long_x):
        
    train_x = [long_x[i] for i in train]
    train_y = [long_time[i] for i in train]
    train_x_len = [all_x_len[i] for i in train]
    #train_static = [long_static[i] for i in train]
    
    train_x, train_y, train_x_len = get_n2n_data(train_x, train_y, train_x_len)
    if len(train_x) % 256 == 1:
        print(len(train_x))
        print('wrong squeeze!')

# for train, test in kfold.split(long_x):
for train, test in kfold.split(long_x):
    if reverse:
        temp = train
        train = test
        test = temp
    
    model = distcare_target(input_dim = input_dim,output_dim=output_dim, d_model=d_model, MHD_num_head=MHD_num_head, d_ff=d_ff, hidden_dim=hidden_dim).to(device)
    
    if transfer_flag:
        checkpoint = torch.load(file_name, \
                        map_location=torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu'))
        pretrain_dict = checkpoint['net']
        model_dict = model.state_dict()
        pretrain_dict = transfer_gru_dict(pretrain_dict, model_dict,latest_idx, common_len)
        model_dict.update(pretrain_dict)
        model.load_state_dict(model_dict)
        
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    fold_count += 1
#     print(train)

    
    train_x = [long_x[i] for i in train]
    train_y = [long_time[i] for i in train]
    train_x_len = [all_x_len[i] for i in train]
    #train_static = [long_static[i] for i in train]
    
    train_x, train_y, train_x_len = get_n2n_data(train_x, train_y, train_x_len)
    
    test_x = [long_x[i] for i in test]
    test_y = [long_time[i] for i in test]
    test_x_len = [all_x_len[i] for i in test]
    #test_static = [long_static[i] for i in test]
    
    test_x, test_y, test_x_len = get_n2n_data(test_x, test_y, test_x_len)
    
    if not os.path.exists('./model/'+data_str):
        os.mkdir('./model/'+data_str)
        
    if transfer_flag:
        target_file_name = './model/'+data_str+'/distcare-trans-'+str(n_splits)+'-fold-LOS-regression' + str(fold_count)#4114
    else:
        target_file_name = './model/'+data_str+'/distcare-no-trans-'+str(n_splits)+'-fold-LOS-regression' + str(fold_count)#4114
    
    fold_train_loss = []
    fold_valid_loss = []
    best_mse = 10000
    best_mad = 0
    best_mape = 0
    best_kappa = 0
    
    for each_epoch in range(epochs):
       
        
        epoch_loss = []
        counter_batch = 0
        model.train()  
        
        for step, (batch_x, batch_y, batch_lens) in enumerate(ckd_batch_iter(train_x, train_y, train_x_len, batch_size, shuffle=True)):  
            optimizer.zero_grad()
            batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
            batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
            batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()

            masks = length_to_mask(batch_lens).unsqueeze(-1).float()

            opt, decov_loss, emb = model(batch_x, batch_lens)

            MSE_Loss = get_re_loss(opt, batch_y.unsqueeze(-1))

#             model_loss = pred_loss + 1e7*decov_loss
            model_loss = MSE_Loss

            loss = model_loss

            epoch_loss.append(MSE_Loss.cpu().detach().numpy())
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 20)
            optimizer.step()
            
            if step % 50 == 0:
                print('Fold %d Epoch %d Batch %d: Train Loss = %.4f'%(fold_count,each_epoch, step, loss.cpu().detach().numpy()))
                logger.info('Fold %d Epoch %d Batch %d: Train Loss = %.4f'%(fold_count,each_epoch, step, loss.cpu().detach().numpy()))
            
        epoch_loss = np.mean(epoch_loss)
        fold_train_loss.append(epoch_loss)

        #Validation
        y_true = []
        y_pred = []
        y_pred_flatten = []
        y_true_flatten = []
        outcome_pred_flatten = []
        outcome_true_flatten = []
        with torch.no_grad():
            model.eval()
            valid_loss = []
            valid_true = []
            valid_pred = []
            for batch_x, batch_y, batch_lens in ckd_batch_iter(test_x, test_y, test_x_len, batch_size):
                batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
                batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
                batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
                masks = length_to_mask(batch_lens).unsqueeze(-1).float()
               
                opt, decov_loss, emb = model(batch_x, batch_lens)
                
                MSE_Loss = get_re_loss(opt, batch_y.unsqueeze(-1))
                
                valid_loss.append(MSE_Loss.cpu().detach().numpy())

                y_pred_flatten += [reverse_los(x, los_info) / 30 for x in list(opt.cpu().detach().numpy().flatten())]
                y_true_flatten += [reverse_los(x, los_info) / 30 for x in list(batch_y.cpu().numpy().flatten())]
            

            valid_loss = np.mean(valid_loss)
            fold_valid_loss.append(valid_loss)
            ret = metrics.print_metrics_regression(y_true_flatten, y_pred_flatten, verbose=0)
            history.append(ret)
            #print()

            if each_epoch % 10 == 0:
                print('Fold %d, epoch %d: Loss = %.4f Valid loss = %.4f MSE = %.4f' % (
                    fold_count, each_epoch, fold_train_loss[-1], fold_valid_loss[-1], ret['mse']), flush=True)
                logger.info('Fold %d, epoch %d: Loss = %.4f Valid loss = %.4f MSE = %.4f' % (
                    fold_count, each_epoch, fold_train_loss[-1], fold_valid_loss[-1], ret['mse']))
                # metrics.print_metrics_regression(y_true_flatten, y_pred_flatten)
                
            cur_mse = ret['mse']
            if cur_mse < best_mse:
                print('------------ Save FOLD-BEST model - MSE: %.4f ------------' % cur_mse, flush=True)
                logger.info('------------ Save FOLD-BEST model - MSE: %.4f ------------' % cur_mse)
                metrics.print_metrics_regression(y_true_flatten, y_pred_flatten)
                best_mse = cur_mse
                best_mad = ret['mad']
                state = {
                    'net': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'epoch': each_epoch
                }
                torch.save(state, target_file_name + '_' + str(fold_count))

                if cur_mse < global_best:
                    global_best = cur_mse
                    state = {
                        'net': model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                        'epoch': each_epoch
                    }
                    torch.save(state, target_file_name)
                    print('------------ Save best model - MSE: %.4f ------------' % cur_mse, flush=True)
                    logger.info('------------ Save best model - MSE: %.4f ------------' % cur_mse)

        print('Fold %d, mse = %.4f, mad = %.4f' % (fold_count, ret['mse'], ret['mad']), flush=True)
        logger.info('Fold %d, mse = %.4f, mad = %.4f' % (fold_count, ret['mse'], ret['mad']))

    mse.append(best_mse)
    mad.append(best_mad)
    total_train_loss.append(fold_train_loss)
    total_valid_loss.append(fold_valid_loss)


print('mse %.4f(%.4f)' % (np.mean(mse), np.std(mse)))
print('mad %.4f(%.4f)' % (np.mean(mad), np.std(mad)))
logger.info('mse %.4f(%.4f)' % (np.mean(mse), np.std(mse)))
logger.info('mad %.4f(%.4f)' % (np.mean(mad), np.std(mad)))

In [ ]:
with open('history.pkl', 'wb') as f:
    pickle.dump(history, f)